In [1]:
# Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install pandas

In [2]:
import json
import pandas as pd
import utils.db_utils as db
import utils.file_utils as file
import utils.bible_utils as bible
import time
from datetime import timedelta

original_words_table = db.original_words_table
target_words_table = db.target_words_table
alignment_table = db.alignment_table
dbPath = './data/alignmentsData.sqlite'
keyTermsPath = './data/keyTerms.json'
alignmentTrainingDataPath = './data/alignmentTrainingData.json'
origLangPathGreek = './data/OrigLangJson/ugnt/v0.14'
origLangPathHebrew = './data/OrigLangJson/uhb/v2.1.15'
targetLangPathEn = './data/TargetLangJson/ult/v14'

connection = db.initAlignmentDB(dbPath)

bibleType = 'en_ult'
testament = 1
dataFolder = './data/Alignments'
bookId = 'tit'

searchOriginal = True
searchTarget = False
searchLemma = True
caseInsensitive = True

Connection to SQLite DB successful


In [7]:
# reading dataFrame from json:
def loadAlignmentDataFromFile(lemma):
    alignment_data_path = f'data/TrainingData/{lemma}.json'
    f = open(alignment_data_path)
    dataStr = f.read()
    data = json.loads(dataStr)
    df = pd.DataFrame(data)
    return df

In [22]:
lemma = 'θεός'
lemmaAlignments = loadAlignmentDataFromFile(lemma)
lemmaAlignments

,id,book_id,chapter,verse,alignment_num,orig_lang_words,target_lang_words,origSpan,origWords,origWordsTxt,alignmentOrigWords,targetSpan,targetWords,targetWordsTxt,alignmentTargetWords,alignmentTxt,frequency
0,118054,luk,1,6,5,",750353,750354,",",727232,",1,"[{'id': 750353, 'book_id': 'luk', 'chapter': '...",τοῦ Θεοῦ,2,0,"[{'id': 727232, 'book_id': 'luk', 'chapter': '...",God,1,τοῦ Θεοῦ = God,0.077381
1,118093,luk,1,8,12,",750396,750397,",",727274,",1,"[{'id': 750396, 'book_id': 'luk', 'chapter': '...",τοῦ Θεοῦ,2,0,"[{'id': 727274, 'book_id': 'luk', 'chapter': '...",God,1,τοῦ Θεοῦ = God,0.077381
2,118206,luk,1,16,8,",750517,750518,",",727452,",1,"[{'id': 750517, 'book_id': 'luk', 'chapter': '...",τὸν Θεὸν,2,0,"[{'id': 727452, 'book_id': 'luk', 'chapter': '...",God,1,τὸν Θεὸν = God,0.043651
3,118262,luk,1,19,11,",750579,750580,",",727539,",1,"[{'id': 750579, 'book_id': 'luk', 'chapter': '...",τοῦ Θεοῦ,2,0,"[{'id': 727539, 'book_id': 'luk', 'chapter': '...",God,1,τοῦ Θεοῦ = God,0.077381
4,118382,luk,1,26,10,",750709,750710,",",727714,",1,"[{'id': 750709, 'book_id': 'luk', 'chapter': '...",τοῦ Θεοῦ,2,0,"[{'id': 727714, 'book_id': 'luk', 'chapter': '...",God,1,τοῦ Θεοῦ = God,0.077381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,165087,2jn,1,3,7,",801721,",",796060,",0,"[{'id': 801721, 'book_id': '2jn', 'chapter': '...",Θεοῦ,1,0,"[{'id': 796060, 'book_id': '2jn', 'chapter': '...",God,1,Θεοῦ = God,0.273810
500,165188,2jn,1,9,9,",801838,",",796230,",0,"[{'id': 801838, 'book_id': '2jn', 'chapter': '...",Θεὸν,1,0,"[{'id': 796230, 'book_id': '2jn', 'chapter': '...",God,1,Θεὸν = God,0.017857
501,165319,3jn,1,6,12,",801998,",",796439,",0,"[{'id': 801998, 'book_id': '3jn', 'chapter': '...",Θεοῦ,1,0,"[{'id': 796439, 'book_id': '3jn', 'chapter': '...",God,1,Θεοῦ = God,0.273810
502,165396,3jn,1,11,11,",802076,802077,",",796557,",1,"[{'id': 802076, 'book_id': '3jn', 'chapter': '...",τοῦ Θεοῦ,2,0,"[{'id': 796557, 'book_id': '3jn', 'chapter': '...",God,1,τοῦ Θεοῦ = God,0.077381


In [29]:
def describeAlignments(alignments):
    descr = alignments.describe()
    print(f"Alignments description: {descr}")

    fields = list(descr.columns)
    print(f"fields = {fields}")

    fields.remove('frequency') # not useful for analysis
    for field in fields:
        alignmentOrigWords_frequency = alignments[field].value_counts()
        print(f"\nFrequency of {field}:\n{alignmentOrigWords_frequency}")

In [30]:
describeAlignments(lemmaAlignments)

Alignments description:          origSpan  alignmentOrigWords  targetSpan  alignmentTargetWords  \
count  504.000000          504.000000  504.000000            504.000000   
mean     0.517857            1.503968    0.319444              1.206349   
std      0.534755            0.508364    1.177632              0.409965   
min      0.000000            1.000000    0.000000              1.000000   
25%      0.000000            1.000000    0.000000              1.000000   
50%      0.500000            1.500000    0.000000              1.000000   
75%      1.000000            2.000000    0.000000              1.000000   
max      2.000000            3.000000   18.000000              3.000000   

        frequency  
count  504.000000  
mean     0.129307  
std      0.098672  
min      0.001984  
25%      0.043651  
50%      0.079365  
75%      0.273810  
max      0.273810  
fields = ['origSpan', 'alignmentOrigWords', 'targetSpan', 'alignmentTargetWords', 'frequency']

Frequency of origSpan:
0